In [3]:
import pandas as pd, numpy as np

In [4]:
path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/tier1234_coverage_data_2021_03_24.csv'
df = pd.read_csv(path)

In [5]:
df.head()

,location_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,estimation_status,source_citation,source_link,data_choice_notes,CI_source
0,Bangladesh,oil,percent of population eating vehicle,na,100.000000,96.514892,100.000000,total population,na,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Keeping Aaron (survey data) number from 2015 (...,modeling
1,Bangladesh,oil,percent of population eating industrially prod...,na,88.400000,84.500000,92.300000,total population,na,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Keeping Aaron number (88%) and descarding Ragh...,extraction
2,Bangladesh,oil,percent of population eating fortified vehicle,vitamin a,42.849572,41.356218,44.342926,NaN,multiplicative,NaN,NaN,NaN,modeling
3,Bangladesh,wheat flour,percent of population eating vehicle,na,41.900000,31.257526,52.542474,women of reproductive age,na,"Leyvraz M, Laillou A, Rahman S, et al. An Asse...",https://www.mdpi.com/2072-6643/8/9/541/htm,"Keeping Levyraz numbers (42% and 76%), as they...",modeling
4,Bangladesh,wheat flour,percent of population eating vehicle,na,76.200000,56.845429,95.554571,under-5,na,"Leyvraz M, Laillou A, Rahman S, et al. An Asse...",https://www.mdpi.com/2072-6643/8/9/541/htm,"Keeping Levyraz numbers (42% and 76%), as they...",modeling


In [6]:
df['data_richness'] = [0 if i=='regression' else 1 for i in df.estimation_status]

In [8]:
output = df.groupby(['location_name','vehicle','value_description','nutrient']).sum()[['data_richness']]

In [9]:
output

data_richness
location_name vehicle     value_description                                  nutrient                 
Bangladesh    oil         percent of population eating fortified vehicle     vitamin a               1
                          percent of population eating industrially produ... na                      1
                          percent of population eating vehicle               na                      1
              wheat flour percent of population eating fortified vehicle     folic acid              1
                                                                             iron                    1
...                                                                                                ...
Viet Nam      wheat flour percent of population eating fortified vehicle     iron                    1
                                                                             vitamin a               1
                                                                             zinc                    1
                          percent of population eating industrially produ... na                      1
                          percent of population eating vehicle               na                      2

[206 rows x 1 columns]

In [11]:
output.sort_values('data_richness')

data_richness
location_name vehicle     value_description                                  nutrient                 
Cameroon      wheat flour percent of population eating fortified vehicle     vitamin a               0
              maize flour percent of population eating fortified vehicle     folic acid              0
                                                                             iron                    0
                                                                             vitamin a               0
                                                                             zinc                    0
...                                                                                                ...
Ethiopia      wheat flour percent of population eating industrially produ... na                      2
                          percent of population eating fortified vehicle     folic acid              2
Burkina Faso  wheat flour percent of population eating vehicle               na                      2
Ethiopia      oil         percent of population eating fortified vehicle     vitamin a               2
Viet Nam      wheat flour percent of population eating vehicle               na                      2

[206 rows x 1 columns]

In [12]:
output = output.reset_index()

In [42]:
C = output[(output.value_description=="percent of population eating fortified vehicle")].drop(columns='value_description').rename(columns={'data_richness':'C'})
B = output[(output.value_description=="percent of population eating industrially produced vehicle")].drop(columns=['value_description','nutrient']).rename(columns={'data_richness':'B'})
A = output[(output.value_description=="percent of population eating vehicle")].drop(columns=['value_description','nutrient']).rename(columns={'data_richness':'A'})

In [47]:
data_rich = C.merge(B, on = ['location_name','vehicle'], how = 'left').merge(A, on = ['location_name','vehicle'], how = 'left')

In [48]:
data_rich[['A','B','C']].drop_duplicates()

,A,B,C
0,1.0,1,1
1,2.0,1,1
7,1.0,1,0
12,2.0,1,0
14,0.0,0,0
18,1.0,0,1
19,2.0,0,1
21,2.0,0,0
41,NaN,1,2
42,NaN,2,2


In [44]:
for col in ['A','B','C']:
    data_rich[col] = ((2 ** data_rich[col]) + 1) % 2

In [45]:
data_rich

,location_name,vehicle,nutrient,C,B,A
0,Bangladesh,oil,vitamin a,1,1,1.0
1,Bangladesh,wheat flour,folic acid,1,1,1.0
2,Bangladesh,wheat flour,iron,1,1,1.0
3,Bangladesh,wheat flour,vitamin a,1,1,1.0
4,Bangladesh,wheat flour,zinc,1,1,1.0
...,...,...,...,...,...,...
117,Viet Nam,oil,vitamin a,0,1,1.0
118,Viet Nam,wheat flour,folic acid,1,1,1.0
119,Viet Nam,wheat flour,iron,1,1,1.0
120,Viet Nam,wheat flour,vitamin a,1,1,1.0


In [33]:
2**2

4

In [34]:
2%%2

SyntaxError: invalid syntax (<ipython-input-34-99f00c324aa1>, line 1)

In [39]:
2 % 2

0

In [36]:
1 % 2

1

In [37]:
3 % 2

1

In [38]:
4 % 2

0